In [1]:
%pip install matplotlib

In [2]:
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

Matplotlib is building the font cache; this may take a moment.


In [3]:

# Funktion zum Parsen der Datei und Extrahieren der Linien
def parse_file(file_path, interval, y_shift, group_size):
    lines = []
    with open(file_path, 'r') as file:
        for line in file:
            # Nummer der Linie extrahieren
            match_num = re.search(r'ADPATH_[LR]_1\[(\d+)\]', line)
            if match_num:
                line_num = int(match_num.group(1))
                # Punkte aus Zeilen extrahieren, die Koordinaten definieren
                matches = re.findall(r'X1\s*([-+]?\d*\.\d+|\d+),\s*Y1\s*([-+]?\d*\.\d+|\d+),\s*X2\s*([-+]?\d*\.\d+|\d+),\s*Y2\s*([-+]?\d*\.\d+|\d+)', line)
                for match in matches:
                    x1, y1, x2, y2 = map(float, match)
                    # Berechne die Verschiebung basierend auf der Nummer der Linie und der Gruppengröße
                    x_shift = interval * ((line_num - 1) % group_size)
                    y_shift_total = y_shift * ((line_num - 1) // group_size)
                    x1 += x_shift
                    x2 += x_shift
                    y1 -= y_shift_total
                    y2 -= y_shift_total
                    lines.append(((x1, y1), (x2, y2)))
    return lines

# Funktion zum Plotten der Linien und Speichern als Vektordatei (PDF)
def plot_lines(lines_l, lines_r):
    with PdfPages('lines_plot.pdf') as pdf:
        plt.figure(dpi=300)  # Erhöhen der Auflösung auf 300 DPI

        for line in lines_l:
            (x1, y1), (x2, y2) = line
            plt.plot([x1, x2], [y1, y2], linewidth=0.80)  # Linien aus pos22_AdPathL.dat zwei Drittel so dick

        for line in lines_r:
            (x1, y1), (x2, y2) = line
            plt.plot([x1, x2], [y1, y2], linewidth=1.0)  # Linien aus pos22_AdPathR.dat normal dick

        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('Lines from pos22_AdPathL.dat and pos22_AdPathR.dat')
        plt.axis('equal')  # Gleicher Maßstab für beide Achsen
        pdf.savefig()  # Speichern als PDF
        plt.close()

# Dateien parsen und Linien extrahieren
file_path_l = 'pos22_AdPathL.dat'
file_path_r = 'pos22_AdPathR.dat'
interval = 265.217391
y_shift = -150

# Setze die Gruppengröße für pos22_AdPathL.dat auf 12 und für pos22_AdPathR.dat auf 13
group_size_l = 13
group_size_r = 13

lines_l = parse_file(file_path_l, interval, y_shift, group_size_l)
lines_r = parse_file(file_path_r, interval, y_shift, group_size_r)

# Extrahierte Linien plotten und als Vektordatei speichern
plot_lines(lines_l, lines_r)

print("Die Vektordatei wurde erfolgreich erstellt und als 'lines_plot.pdf' gespeichert.")

Die Vektordatei wurde erfolgreich erstellt und als 'lines_plot.pdf' gespeichert.
